# 参数管理

我们首先看一下具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.5918],
        [-0.8349]], grad_fn=<AddmmBackward0>)

参数访问

In [2]:
print(net[2].state_dict()) # state_dict()相当于一个Oderedlist，里面包含了每一层的参数

OrderedDict([('weight', tensor([[ 0.3016, -0.1901, -0.1991, -0.1220,  0.1121, -0.1424, -0.3060,  0.3400]])), ('bias', tensor([-0.0291]))])


目标参数

In [5]:
print(type(net[2].bias))  # bias是一个Parameter类型,是一个特殊的Tensor,可以优化
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad) # 梯度为None,因为还没有进行反向传播

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2068], requires_grad=True)
tensor([-0.2068])
None


In [6]:
net[2].weight.grad == None

True

一次性访问所有参数

In [7]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [10]:
print(net.state_dict()['2.bias'].data)

# state_dict()只包含可学习的参数，而且每个参数都是一个Tensor
# 优化器optimizer的state_dict()包含关于优化器状态信息的内容
# 例如，optimizer的momentum和参数的学习率
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'foreach': None,
   'differentiable': False,
   'params': [0, 1, 2, 3]}]}

从嵌套块收集参数

In [12]:
def block1():
    return nn.Sequential(   nn.Linear(4, 8), nn.ReLU(),
                            nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # add_module的第一个参数是一个字符串，它将成为子模块的名称, 这是他跟放入Sequential的唯一区别
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2002],
        [-0.2002]], grad_fn=<AddmmBackward0>)

设计了网络后，我们看看它是如何工作的

In [13]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [19]:
rgnet[0][1][0].bias.data

tensor([-0.3645,  0.2492,  0.0348,  0.2790,  0.0483, -0.1807, -0.1325,  0.3204])

内置初始化

In [20]:
def init_normal(m):
    if type(m) == nn.Linear:
        # nn.init 是一个初始化模块,它提供了多种预设的初始化方法
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)  # .apply()会递归地搜索网络内的所有模块，将其作为参数传到init_normal函数中
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0088, -0.0013, -0.0008,  0.0018]), tensor(0.))

In [21]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) # 肯定不能把所有的参数都初始化为1
        nn.init.zeros_(m.bias) # 也可以直接用m.bias.data.fill_(0)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法

In [22]:
def init_xavier(m):
    if type(m) == nn.Linear:
        # nn.init.xavier_uniform_() 会根据该层输入和输出的维度自动设置每一层的权重参数的初始化分布
        nn.init.xavier_uniform_(m.weight) # Xavier均匀分布
        # nn.init.xavier_normal_() Xavier正态分布
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) # 宇宙的终极答案

net[0].apply(init_xavier)    # 只初始化第一层
net[2].apply(init_42)        # 只初始化第三层
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2564, -0.1572, -0.1162,  0.6347])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [25]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5  # 仅保留绝对值大于等于5的权重

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.4824,  9.1729,  0.0000, -0.0000],
        [ 7.6849,  7.6811, -6.7339,  8.1005]], grad_fn=<SliceBackward0>)

In [26]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, 10.1729,  1.0000,  1.0000])

参数绑定

In [28]:
shared = nn.Linear(8, 8)  # 使用相同的参数初始化两个全连接层
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])  # 由于模型参数是共享的，因此改变其中一个的值会影响另外一个

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
